In [2]:
import pandas as pd

In [1]:
from google.colab import drive
drive.mount('/content/drive')
way = "/content/drive/MyDrive/Colab_Notebooks/aaa_ml/datasets/"

Mounted at /content/drive


In [3]:
df = pd.read_parquet(way + "mlsd-eda.parquet")
df.head(5)

,event_date,eid,x_eid,user_id,item_id,microcat_id
0,2023-04-16 23:59:59,301,300,2010890,1435852750630,3674
1,2023-04-16 23:59:59,301,300,379294501,1437733502707,1673
2,2023-04-16 23:59:59,301,300,198428250055,1444440250530,3831
3,2023-04-16 23:59:59,301,2012,705812500004,1444999750230,347447750001
4,2023-04-16 23:59:59,301,2012,1116509750025,1378823251390,1804


Расшифровка полей:
eid (id события)


* 301	просмотр
* 303	просмотр номера телефона (является контактом)
* 308	шеринг в соц. сетях.
* 401	добавление в избранное
* 857	отправка сообщения (является контактом)



x_eid (id источника, откуда произошло событие)



* 300	поисковая выдача
* 400	список избранного
* 2012	главная страница (рекомендации)
* 2017	профиль продавца
* 2437	похожие объявления на карточке товара

In [8]:
df1 = df[df["eid"] == 401]
df1['date'] = df1['event_date'].dt.date

df1.groupby(["user_id", "date"]).agg({"item_id": "nunique"}).sort_values("item_id")

<ipython-input-8-924e161a6c85>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['date'] = df1['event_date'].dt.date


item_id
user_id       date               
8             2023-04-11        1
620258500037  2023-04-14        1
              2023-04-13        1
              2023-04-11        1
620258250072  2023-04-11        1
...                           ...
1191633250840 2023-04-12      267
1186679000797 2023-04-15      279
765918750003  2023-04-15      334
305849250203  2023-04-11      491
765918750003  2023-04-13      833

[537889 rows x 1 columns]

Какая доля объявлений имеет только просмотры (301 событие)? Напишите ответ от 0 до 1 с точностью до 2 знака после запятой.

In [11]:
df2 = df.groupby("item_id").agg({"eid": "nunique"})

len1 = df2[df2["eid"] == 1].shape[0]
len2 = df2.shape[0]
print(f'{len1} / {len2} = {len1/len2}')

578324 / 948602 = 0.6096592670055513


В какой час пользователи больше всего делятся объявлениями в соцсетях (308 событие) ? В ответ напишите целое число от 0 до 23.

In [12]:
df3 = df[df["eid"] == 308]
df3['event_date'] = pd.to_datetime(df3['event_date'])
df3['event_hour'] = df3['event_date'].dt.hour

df3.groupby("event_hour")["eid"].count()

<ipython-input-12-bdd0a39f3e22>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['event_date'] = pd.to_datetime(df3['event_date'])
<ipython-input-12-bdd0a39f3e22>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['event_hour'] = df3['event_date'].dt.hour


event_hour
0      1033
1       925
2      1782
3      3693
4      6335
5      9407
6     11021
7     11912
8     12375
9     12207
10    12470
11    12005
12    12025
13    11425
14    11882
15    12566
16    12813
17    13439
18    12736
19     9913
20     6238
21     3642
22     2437
23     1683
Name: eid, dtype: int64

В какой день недели доля контактов (303 и 857) с главной страницы (2012) от всех контактов наибольшая?

In [20]:
df4 = df[df["eid"].isin([303, 857])]
df4['date'] = df4['event_date'].dt.date
df4['is_main'] = df4['x_eid'].apply(lambda x: x in [2012])
df4.groupby("date")["is_main"].mean()

<ipython-input-20-c38bc0844661>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4['date'] = df4['event_date'].dt.date
<ipython-input-20-c38bc0844661>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4['is_main'] = df4['x_eid'].apply(lambda x: x in [2012])


date
2023-04-10    0.231337
2023-04-11    0.230483
2023-04-12    0.230626
2023-04-13    0.227097
2023-04-14    0.233551
2023-04-15    0.235977
2023-04-16    0.244411
Name: is_main, dtype: float64

Сколько минут длилась самая длинная по времени сессия? Сессия - такая последовательность событий одного пользователя, что между двумя соседними по времени событиями разница не более 10 минут. Длина сессии - разность между первым и последним событием. Дайте ответ в минутах (секунды просто отбросить, не округлять).



In [22]:
df5 = df.sort_values(by=['user_id', 'event_date'])

df5['time_diff'] = df5.groupby('user_id')['event_date'].diff()
df5['session_id'] = (df5['time_diff'] > pd.Timedelta(minutes=10)).cumsum()

session_start_end = df5.groupby(['user_id', 'session_id'])['event_date'].agg(['min', 'max'])

session_start_end['session_duration'] = (session_start_end['max'] - session_start_end['min']).dt.total_seconds() / 60
session_start_end['session_duration'].max()

308.25

Сколько пользователей чаще пользуются рекомендациями (источники действий - 2012 и 2437), чем поиском (300)? В ответ напишите целое число - количество пользователей.

In [28]:
def rec_vs_search(x):
  if x == 2012 or x == 2437:
    return 1
  if x == 300:
    return -1
  return 0

df6 = df.copy()
df6["rec_search"] = df["x_eid"].apply(rec_vs_search)
df6 = df6.groupby(["user_id"]).agg({"rec_search": "sum"})
df6[df6["rec_search"] > 0].shape

(750925, 1)

Какая микрокатегория имеет наибольшее среднее количество действий на объявление? Рассматриваем только микрокатегории, где более 100 уникальных объявлений. В ответ запишите microcat_id.

In [32]:
df7 = df.groupby("microcat_id").agg({"item_id": "nunique", "event_date": "count"})
df7 = df7[df7["item_id"] > 100]
df7["res"] = df7["event_date"] / df7["item_id"]
df7.sort_values("res")

,item_id,event_date,res
microcat_id,,,
89,4474,9763,2.182164
10750010,3823,11089,2.900602
5096,227,716,3.154185
921,710,2244,3.160563
1189,226,726,3.212389
...,...,...,...
50,122,28887,236.778689
501822,121,38607,319.066116
3473,103,33177,322.106796


Какая доля действий в приходится на самое популярное (вообще) объявление в его микрокатегории? Напишите ответ от 0 до 1 с точностью до 2 знака после запятой.
Пример: у самого популярного объявления 100 действий, а в его мирокатегории - 1000. Значит ответ должен быть 0.1

In [36]:
df8 = df.groupby("item_id").agg({"event_date": "count"})
df8["event_date"].sort_values()

item_id
1247261000326        1
1272301001252        1
1423701001880        1
1423701001906        1
1423701002488        1
                 ...  
1443330750388    11104
1442952250421    11674
1443006503581    14794
1440560501403    15407
1433680000132    57097
Name: event_date, Length: 948602, dtype: int64

In [39]:
df81 = df[df["item_id"] == 1433680000132]
df81.head(1)

,event_date,eid,x_eid,user_id,item_id,microcat_id
221,2023-04-16 23:59:34,301,2012,1095679000071,1433680000132,3250006


In [40]:
df82 = df[df["microcat_id"] == 3250006]
57097 / df82.shape[0]

0.9237651474704331

Какая микрокатегория наиболее похожа на микрокатегорию 259? Похожесть определяется так: количество пользователей, которые смотрели исходную и целевую мирокатегорию вместе (за всё время). В ответ запишите microcat_id.

In [41]:
target_microcat = 259

users_watched_target = df[df['microcat_id'] == target_microcat]['user_id'].unique()
df9 = df[df['user_id'].isin(users_watched_target)]

df91 = df9.groupby('microcat_id').size()
df91 = df91.drop(target_microcat)
df91.idxmax()

6702

определить город

In [42]:
df["user_id"].nunique()

1803790